### Average meshes


In [3]:
import numpy as np
import tripy
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt

def load_obj(file_path):
    vertices = []
    faces = []

    with open(file_path, 'r') as file:
        for line in file:
            tokens = line.strip().split()
            if not tokens:
                continue
            if tokens[0] == 'v':
                vertices.append([float(tokens[1]), float(tokens[2]), float(tokens[3])])
            elif tokens[0] == 'f':
                faces.append([int(vertex.split('/')[0]) - 1 for vertex in tokens[1:]])

    return np.array(vertices), np.array(faces)

def save_obj(file_path, vertices, faces):
    with open(file_path, 'w') as file:
        for vertex in vertices:
            file.write(f'v {vertex[0]} {vertex[1]} {vertex[2]}\n')
        for face in faces:
            file.write(f'f {" ".join(str(idx + 1) for idx in face)}\n')

def average_meshes(obj_file1, obj_file2, output_file):
    vertices1, faces1 = load_obj(obj_file1)
    vertices2, faces2 = load_obj(obj_file2)

    # Step 2: Calculate Averaged Vertices
    averaged_vertices = (vertices1 + vertices2) / 2.0

    # Step 3: Create Averaged Mesh
    averaged_mesh_faces = faces1  # Use faces from one of the original meshes

    # Step 4: Save Averaged Mesh to a new OBJ file
    save_obj(output_file, averaged_vertices, averaged_mesh_faces)

# Example usage:
# obj_file1 = r"C:\Users\gabri\OneDrive\Documentos\GitHub\HRN\assets\examples\test_images_results_2\test4\test4_0_hrn_high_mesh.obj"
# obj_file2 = r"C:\Users\gabri\OneDrive\Documentos\GitHub\HRN\assets\examples\test_images_results_2\test4\test4_0_hrn_mid_mesh.obj"
    

obj_file1 = r"C:\Users\gabri\OneDrive\Documentos\GitHub\HRN\assets\examples\tmultiview_webcam_results\test_view_1_0_hrn_high_mesh.obj"
obj_file2 = r"C:\Users\gabri\OneDrive\Documentos\GitHub\HRN\assets\examples\tmultiview_webcam_results\test_view_2_0_hrn_high_mesh.obj"

output_file = r"C:\Users\gabri\OneDrive\Documentos\GitHub\HRN\assets\examples\averaged_test_webcam.obj"


average_meshes(obj_file1, obj_file2, output_file)


In [ ]:
import numpy as np
import tripy
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from PIL import Image

def load_obj(file_path):
    vertices = []
    faces = []

    with open(file_path, 'r') as file:
        for line in file:
            tokens = line.strip().split()
            if not tokens:
                continue
            if tokens[0] == 'v':
                vertices.append([float(tokens[1]), float(tokens[2]), float(tokens[3])])
            elif tokens[0] == 'f':
                faces.append([int(vertex.split('/')[0]) - 1 for vertex in tokens[1:]])

    return np.array(vertices), np.array(faces)

def save_obj(file_path, vertices, faces):
    with open(file_path, 'w') as file:
        for vertex in vertices:
            file.write(f'v {vertex[0]} {vertex[1]} {vertex[2]}\n')
        for face in faces:
            file.write(f'f {" ".join(str(idx + 1) for idx in face)}\n')

def average_images(image_paths):
    # Load images and convert to NumPy arrays
    images = [np.array(Image.open(path)) for path in image_paths]

    # Calculate average image
    averaged_image = np.mean(images, axis=0).astype(np.uint8)

    return averaged_image

def map_image_to_mesh(mesh_vertices, image, output_file):
    # Assuming mesh_vertices is the averaged vertices from the previous step

    # Save the averaged image to a file (optional, for visualization purposes)
    plt.imshow(image)
    plt.savefig('averaged_image.png')
    plt.close()

    # You'll need to implement the logic to map the image onto the 3D mesh using the UV mapping coordinates.
    # This might involve interpolating the image values for each vertex based on their UV coordinates.

    # Step 3: Save the result to a new OBJ file
    save_obj(output_file, mesh_vertices, averaged_mesh_faces)

# Example usage:
obj_file1 = 'path/to/mesh1.obj'
obj_file2 = 'path/to/mesh2.obj'
image_paths = ['path/to/image1.png', 'path/to/image2.png']
output_file = 'path/to/averaged_mesh.obj'

# Step 1: Load Meshes
vertices1, faces1 = load_obj(obj_file1)
vertices2, faces2 = load_obj(obj_file2)

# Step 2: Calculate Averaged Vertices
averaged_vertices = (vertices1 + vertices2) / 2.0

# Step 2a: Average Images
averaged_image = average_images(image_paths)

# Step 3: Map Averaged Image to Averaged Mesh
map_image_to_mesh(averaged_vertices, averaged_image, output_file)


### Calculate ICP 

In [ ]:
import open3d as o3d

def load_obj(file_path):
    mesh = o3d.io.read_triangle_mesh(file_path)
    vertices = np.asarray(mesh.vertices)
    return vertices

def save_obj(file_path, vertices):
    mesh = o3d.geometry.TriangleMesh()
    mesh.vertices = o3d.utility.Vector3dVector(vertices)
    o3d.io.write_triangle_mesh(file_path, mesh)

def perform_icp(source_file, target_file, output_file, max_iterations=100, tolerance=1e-5):
    # Load meshes
    source_vertices = load_obj(source_file)
    target_vertices = load_obj(target_file)

    # Convert to open3d point clouds
    source_cloud = o3d.geometry.PointCloud()
    source_cloud.points = o3d.utility.Vector3dVector(source_vertices)

    target_cloud = o3d.geometry.PointCloud()
    target_cloud.points = o3d.utility.Vector3dVector(target_vertices)

    # Perform ICP
    icp_result = o3d.pipelines.registration.registration_icp(
        source_cloud, target_cloud, max_correspondence_distance=0.05,
        estimation_method=o3d.pipelines.registration.TransformationEstimationPointToPoint(),
        criteria=o3d.pipelines.registration.ICPConvergenceCriteria(max_iteration=max_iterations, relative_fitness=tolerance, relative_rmse=tolerance)
    )

    # Apply the transformation to the source mesh
    source_cloud.transform(icp_result.transformation)
    transformed_vertices = np.asarray(source_cloud.points)

    # Save the transformed mesh to a new OBJ file
    save_obj(output_file, transformed_vertices)

    return icp_result.transformation

# Example usage:
source_obj_file = 'path/to/source_mesh.obj'
target_obj_file = 'path/to/target_mesh.obj'
output_obj_file = 'path/to/transformed_mesh.obj'

transformation_matrix = perform_icp(source_obj_file, target_obj_file, output_obj_file)

print("Transformation Matrix:")
print(transformation_matrix)
